### Purpose

For the analysis we need to construct some phenotypic and covariant data. This notebook loads

* PCS's from the MRC IEU UK Biobank file
* Removal ID's, so we exclude those whom have removed consent
* A linker between our application of 41382 to the IEU common
* Data extracted from application 41382 on BMI, gender and YoB

Then combines the PC's from the IEU to your application, and converts our IID's to the IID type of the genetic file

In [26]:
from miscSupports import terminal_time, flip_list
from csvObject import CsvObject, write_csv
import statsmodels.formula.api as smf
from pathlib import Path
import pandas as pd

common_path = r"Z:\UKB\GeographicID\Paper Data Extraction\Construction Files"
project_path = r"Z:\UKB\GeographicID\Paper Data Extraction\SB_Papers\SW_GWAS"

print(f"Set Environment {terminal_time()}")

Set Environment 10:38


In [2]:
# Load the principle components, set them as IID: [PC1, PC2, ... PC(N)]
pca_file = CsvObject(Path(common_path, "IEU_PCs_1_40.txt"), file_headers=False)
pc_dict = {row[0]: [r for r in row[2:] if r != ""] for row in pca_file.row_data}

# Load removal file for exclusion on IID
removal_ids = CsvObject(Path(common_path, "UKB_Withdrawal_IDs.csv"), set_columns=True)

# Set a linker between 41382: IEU Common
linker = {app: ieu for ieu, app in CsvObject(Path(common_path, "Linker.csv")).row_data}

# Load phenotype (BMI), gender, and year of birth from the data extraction from 41382
variables = CsvObject(Path(project_path, "Variables.csv"))

print(f"Loaded required files {terminal_time()}")

Loaded required files 9:59


In [3]:
analysis_rows = []
for iid, gender, yob, phenotype in variables.row_data:
    
    # If the iid is not set as withdrawn and is within the linker file between our applications
    if (iid not in removal_ids[0]) and iid in linker.keys():
        # Extract the PC via the linker and then append to the output container
        pcs = pc_dict[linker[iid]]
        analysis_rows.append([linker[iid], linker[iid], phenotype, gender, yob] + pcs)
        
print(f"Set IID file {terminal_time()}")

Set IID file 9:59


In [4]:
# Validate header length == row length
headers = ["FID", "IID", "BMI", "Gender", "YoB"] + [f"PC{i}" for i in range(1, 41)]
print(f"Header length is {len(headers)}: Row length is {len(analysis_rows[0])}")

Header length is 45: Row length is 45


In [5]:
# Write the file to disk
write_csv(project_path, "Analysis", headers, analysis_rows)
print(f"Output written at {terminal_time()}")

Output written at 10:9


### Create phenotypic residuals

For several of our analysis runs we will need the residualised phenotype, so we generate that here

In [9]:
# Load analysis sample
analysis = pd.read_csv(Path(project_path, "Analysis.csv"))
print(f"Set analysis data-frame {terminal_time()}")

Set analysis data-frame 10:20


In [20]:
# Set the formula as phenotype ~ all other explanatory variables then run the OLS
formula = "BMI~" + "+".join([h for h in analysis.columns[3:]])
result = smf.ols(formula=formula, data=analysis)
print(f"Set residualised phenotype {terminal_time()}")

Set residualised phenotype 10:26


In [30]:
print(result.fit().resid)


0          8.151475
1         10.910054
2         -2.538434
3         -1.543807
4         -1.977236
            ...    
488237     1.600320
488238    -3.640768
488239     6.406163
488240    -0.215032
488241     1.195356
Length: 486262, dtype: float64
0         IEU6498273
1         IEU2766806
2         IEU4946230
3         IEU6907277
4         IEU4159466
             ...    
488237    IEU3797450
488238    IEU6382416
488239    IEU5291816
488240    IEU7325037
488241    IEU6604467
Name: FID, Length: 488242, dtype: object
